# Data Collection from Scratch

To get a real understanding of the components, we'll set aside convenience functions and configuration files, building everything from scratch to see how it all fits together.

### Make a Run Engine

In [0]:
from bluesky.run_engine import RunEngine

In [0]:
metadata = {'owner': 'demo', 'group': 'demo', 'beamline_id': 'demo'}  # bare minimum required metadata
RE = RunEngine(metadata)

### Use a simulated motor and detector

We'll take a closer look at these below, and we'll learn how to define new ones.

In [0]:
from bluesky.examples import motor, det

### Use a plan

Below, we'll take a closer look at exactly what this object is and does.

In [0]:
from bluesky.scans import AbsScan

Like most one-dimensional scans, `AbsScan` takes a list of detectors and a motor. It also takes the `start`, `stop`, and `num`, the number of points.

In [0]:
plan = AbsScan([det], motor, 1, 5, 5)

### Execute the plan

Pass the plan to the `RunEngine` to run it.

In [0]:
RE(plan)

### Get live feedback

The `RunEngine` accepts a second arguments, `subs`, short for "subscriptions". These are functions that subscribe to information generated by the scan. The functions are called "callbacks." Some basic callbacks are included with bluesky. You can also write your own.

In [0]:
from bluesky.callbacks import LiveTable, LivePlot

In [0]:
RE(plan, LiveTable(['motor', 'det']))

Let's add a plot as well.

In [0]:
# This line is needed to make plots show in the notebook.
# If you not in the notebook, it does not apply and will not work.
%matplotlib notebook

In [0]:
RE(plan, [LiveTable(['motor', 'det']), LivePlot('det')])

Notice that the `RunEngine` accepts a single callable like `LiveTable(...)` or a list of callables like [`LiveTable(...), LivePlot(...)]`.

We can alter the attributes of our plan. For example, we can increase the number of points.

In [0]:
plan.num = 10

Observe that this will add a new line to our current plot, above.

In [0]:
RE(plan, LiveTable(['motor', 'det']))

## What exactly is `plan`?

It is an iterable -- loosely speaking a list -- of Messages, granular instructions to be executed by the RunEngine. Between each instruction, the RunEngine intervenes to handle interruptions and bundling up data into Documents.

Let's make a simple plan from scratch.

In [0]:
from bluesky import Msg

RE([Msg('open_run'),
    Msg('close_run')],
   LiveTable(['motor', 'det']))

In [0]:
RE([Msg('open_run'),
    Msg('create'),  # bundle future readings into an Event Document
    Msg('set', motor, 5),
    Msg('read', motor),
    Msg('read', det),
    Msg('save'),  # close and save bundle
    Msg('close_run')],
   LiveTable(['motor', 'det']))

We can use a for loop to compose Messages into a plan.

In [0]:
p = []
p.append(Msg('open_run'))
for position in [1, 2, 3]:
    p.append(Msg('create'))
    p.append(Msg('set', motor, position))
    p.append(Msg('read', motor))
    p.append(Msg('read', det))
    p.append(Msg('save'))
p.append(Msg('close_run'))

RE(p, LiveTable(['motor', 'det']))

Or, equivalently, we can generate messages on the fly using a generator.

In [0]:
def plan_generator():
    yield Msg('open_run')
    for position in [1, 2, 3]:
        yield Msg('create')
        yield Msg('set', motor, position)
        yield Msg('read', motor)
        yield Msg('read', det)
        yield Msg('save')
    yield Msg('close_run')

RE(plan_generator(), LiveTable(['motor', 'det']))

These toy examples omit a couple key steps -- triggering the detector, waiting for the motors to move into position, and so on. When all the details are included, the list of Messages becomes lengthy.

To address this, classes like ``AbsScan`` automate the message-writing. They provide a powerful "middle layer," balancing flexibility with ease of use.

In [0]:
list(AbsScan([det], motor, 1, 2, 2))